# TP : Natural Language Processing (NLP) avec Transformers

## Objectifs
- Comprendre l'architecture des Transformers et leur importance en NLP
- Utiliser des modèles pré-entraînés (BERT, DistilBERT, etc.) avec Hugging Face
- Fine-tuner des transformers pour des tâches spécifiques
- Implémenter des applications NLP : analyse de sentiment, classification de texte
- Comparer les transformers avec les approches classiques (RNN, LSTM)
- Comprendre les concepts de tokenization, attention, et embeddings


## 1. Imports et Configuration

Importation des bibliothèques nécessaires pour travailler avec les transformers.


In [ ]:
# Transformers et NLP
from transformers import (
    AutoTokenizer, AutoModel, AutoModelForSequenceClassification,
    pipeline, Trainer, TrainingArguments,
    BertTokenizer, BertModel,
    DistilBertTokenizer, DistilBertForSequenceClassification
)
from transformers import AdamW, get_linear_schedule_with_warmup

# Deep Learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Data manipulation
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_STATE)
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Style pour les graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")


## 2. Introduction aux Transformers

### 2.1. Concepts Fondamentaux

Les **Transformers** ont révolutionné le NLP moderne avec l'introduction du mécanisme d'**Attention**.

**Architecture Transformer :**
- **Encoder-Decoder** : Architecture originale (traduction)
- **Encoder-only** : BERT, DistilBERT (compréhension du langage)
- **Decoder-only** : GPT (génération de texte)

**Composants clés :**
1. **Self-Attention** : Permet au modèle de se concentrer sur différentes parties de l'input
2. **Multi-Head Attention** : Plusieurs têtes d'attention en parallèle
3. **Feed-Forward Networks** : Réseaux fully connected
4. **Layer Normalization** : Stabilisation de l'entraînement
5. **Positional Encoding** : Encodage de la position des mots

**Avantages des Transformers :**
- Parallélisation (vs RNN séquentiels)
- Long-range dependencies (meilleur que RNN/LSTM)
- Modèles pré-entraînés puissants (BERT, GPT, etc.)
- Transfer learning efficace

### 2.2. Modèles Pré-entraînés Populaires

- **BERT** (Bidirectional Encoder Representations from Transformers) : Compréhension bidirectionnelle
- **DistilBERT** : Version légère et rapide de BERT
- **RoBERTa** : Optimisation de BERT
- **GPT** : Génération de texte
- **T5** : Text-to-Text Transfer Transformer


## 3. Chargement et Préparation des Données

### 3.1. Dataset d'Analyse de Sentiment

Nous utiliserons un dataset d'analyse de sentiment pour démontrer l'utilisation des transformers.


In [ ]:
# Option 1: Utiliser un dataset depuis Hugging Face datasets
try:
    from datasets import load_dataset
    print("Chargement du dataset IMDB depuis Hugging Face...")
    dataset = load_dataset("imdb", split="train[:5000]+test[:1000]")  # Échantillon pour la démo
    df = dataset.to_pandas()
    df = df[['text', 'label']]
    df.columns = ['text', 'sentiment']
    print(f"Dataset chargé: {len(df)} échantillons")
    DATASET_LOADED = True
except Exception as e:
    print(f"Impossible de charger depuis Hugging Face: {e}")
    print("Création d'un dataset d'exemple...")
    # Dataset d'exemple simple
    texts = [
        "This movie is absolutely fantastic! I loved every minute of it.",
        "Terrible film, complete waste of time. Boring and poorly acted.",
        "Great acting and a compelling storyline. Highly recommended!",
        "The worst movie I've ever seen. Plot makes no sense.",
        "Excellent cinematography and brilliant performances.",
        "I found this movie quite boring and predictable.",
        "Outstanding! One of the best films this year.",
        "Poor direction and bad script. Very disappointed.",
    ] * 100  # Répéter pour avoir plus de données
    labels = [1, 0, 1, 0, 1, 0, 1, 0] * 100
    df = pd.DataFrame({'text': texts, 'sentiment': labels})
    DATASET_LOADED = False

# Afficher quelques exemples
print("\nPremiers exemples du dataset:")
print(df.head(10))
print(f"\nDistribution des sentiments:")
print(df['sentiment'].value_counts())
print(f"\nExemples de textes:")
for i in range(3):
    print(f"\n{i+1}. Sentiment: {df.iloc[i]['sentiment']} (1=positif, 0=négatif)")
    print(f"   Texte: {df.iloc[i]['text'][:100]}...")


### 3.2. Préparation des Données

Division en ensembles d'entraînement, validation et test.


In [ ]:
# Division train/validation/test
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    df['text'].values, df['sentiment'].values, 
    test_size=0.3, random_state=RANDOM_STATE, stratify=df['sentiment']
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, 
    test_size=0.5, random_state=RANDOM_STATE, stratify=temp_labels
)

print(f"Train set: {len(train_texts)} échantillons")
print(f"Validation set: {len(val_texts)} échantillons")
print(f"Test set: {len(test_texts)} échantillons")
print(f"\nDistribution des sentiments (train):")
print(pd.Series(train_labels).value_counts().sort_index())


## 4. Utilisation des Transformers avec Hugging Face

### 4.1. Pipeline Simple (Zero-Shot Classification)

La façon la plus simple d'utiliser les transformers est via les **pipelines** de Hugging Face.


In [ ]:
# Pipeline pour l'analyse de sentiment (pré-entraîné)
print("Chargement du pipeline d'analyse de sentiment...")
sentiment_pipeline = pipeline("sentiment-analysis", device=0 if torch.cuda.is_available() else -1)

# Tester sur quelques exemples
test_sentences = [
    "I love this movie! It's fantastic!",
    "This is the worst film I've ever seen.",
    "The movie was okay, nothing special.",
    "Brilliant acting and excellent cinematography!"
]

print("\n" + "="*70)
print("ANALYSE DE SENTIMENT AVEC PIPELINE")
print("="*70)

for sentence in test_sentences:
    result = sentiment_pipeline(sentence)[0]
    print(f"\nTexte: {sentence}")
    print(f"Label: {result['label']}")
    print(f"Score: {result['score']:.4f}")


### 4.2. Tokenization avec BERT

La tokenization est cruciale pour les transformers. Explorons comment BERT tokenise le texte.


In [ ]:
# Charger le tokenizer BERT
print("Chargement du tokenizer BERT...")
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Exemple de tokenization
sample_text = "I love transformers and deep learning!"
print(f"\nTexte original: {sample_text}")
print(f"Longueur du texte: {len(sample_text.split())} mots")

# Tokenization
tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
encoded = tokenizer.encode(sample_text, add_special_tokens=True)

print(f"\nTokens: {tokens}")
print(f"Nombre de tokens: {len(tokens)}")
print(f"\nToken IDs: {token_ids}")
print(f"\nEncodage complet (avec [CLS] et [SEP]): {encoded}")
print(f"Longueur encodée: {len(encoded)}")

# Décodage
decoded = tokenizer.decode(encoded)
print(f"\nDécodé: {decoded}")

# Visualisation des tokens spéciaux
print("\n" + "="*70)
print("TOKENS SPÉCIAUX BERT")
print("="*70)
print(f"[CLS]: {tokenizer.cls_token_id} (début de séquence)")
print(f"[SEP]: {tokenizer.sep_token_id} (fin de séquence)")
print(f"[PAD]: {tokenizer.pad_token_id} (padding)")
print(f"[UNK]: {tokenizer.unk_token_id} (token inconnu)")

# Tokenization d'un batch avec padding et truncation
texts_batch = [
    "This is a short text.",
    "This is a much longer text that needs to be handled properly with padding and truncation to fit the model's requirements."
]

encoded_batch = tokenizer(
    texts_batch,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

print("\n" + "="*70)
print("TOKENIZATION DE BATCH")
print("="*70)
print(f"Input IDs shape: {encoded_batch['input_ids'].shape}")
print(f"Attention mask shape: {encoded_batch['attention_mask'].shape}")
print(f"\nInput IDs:")
print(encoded_batch['input_ids'])
print(f"\nAttention mask (1 = vrai token, 0 = padding):")
print(encoded_batch['attention_mask'])


## 5. Fine-tuning d'un Transformer pour Classification

### 5.1. Création d'un Dataset Personnalisé

Créons une classe Dataset pour gérer nos données efficacement.


In [ ]:
class SentimentDataset(Dataset):
    """Dataset personnalisé pour l'analyse de sentiment"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenization
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Utiliser DistilBERT (plus léger et rapide que BERT)
print("Chargement de DistilBERT...")
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

# Créer les datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer)

print(f"\nTrain dataset: {len(train_dataset)} échantillons")
print(f"Validation dataset: {len(val_dataset)} échantillons")
print(f"Test dataset: {len(test_dataset)} échantillons")

# Visualiser un exemple
sample = train_dataset[0]
print(f"\nExemple du dataset:")
print(f"Input IDs shape: {sample['input_ids'].shape}")
print(f"Attention mask shape: {sample['attention_mask'].shape}")
print(f"Label: {sample['label'].item()}")
print(f"Texte original: {train_texts[0][:100]}...")


### 5.2. Chargement du Modèle Pré-entraîné

Chargeons DistilBERT pour la classification de séquences.


In [ ]:
# Charger le modèle pré-entraîné pour classification
num_labels = 2  # Classification binaire (positif/négatif)

model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels
)

model = model.to(device)
print(f"Modèle chargé sur {device}")
print(f"Nombre de paramètres: {sum(p.numel() for p in model.parameters()):,}")

# Afficher l'architecture du modèle
print("\nArchitecture du modèle:")
print(model)


### 5.3. Fonction d'Entraînement

Créons les fonctions nécessaires pour l'entraînement.


In [ ]:
# Configuration de l'entraînement
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Optimizer et scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_loader) * NUM_EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Nombre d'epochs: {NUM_EPOCHS}")
print(f"Total steps: {total_steps}")
print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")


In [ ]:
def train_epoch(model, dataloader, optimizer, scheduler, device):
    """Entraîne le modèle pour une époque"""
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        # Métriques
        total_loss += loss.item()
        predictions = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(predictions == labels)
        total_samples += labels.size(0)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions.double() / total_samples
    
    return avg_loss, accuracy.item()

def eval_model(model, dataloader, device):
    """Évalue le modèle"""
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            logits = outputs.logits
            
            total_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(predictions == labels)
            total_samples += labels.size(0)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions.double() / total_samples
    
    return avg_loss, accuracy.item(), all_predictions, all_labels

print("Fonctions d'entraînement et d'évaluation définies.")


### 5.4. Entraînement du Modèle

Lançons l'entraînement du modèle.


In [ ]:
# Historique de l'entraînement
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print("Démarrage de l'entraînement...")
print("="*70)

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    print("-" * 70)
    
    # Entraînement
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, device)
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    
    # Validation
    val_loss, val_acc, _, _ = eval_model(model, val_loader, device)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

print("\n" + "="*70)
print("Entraînement terminé!")
print("="*70)


### 5.5. Visualisation de l'Entraînement

Visualisons les courbes d'apprentissage.


In [ ]:
# Visualisation de l'historique
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2, marker='o')
axes[0].plot(history['val_loss'], label='Validation Loss', linewidth=2, marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2, marker='o')
axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2, marker='s')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Meilleure validation accuracy: {max(history['val_acc']):.4f}")
print(f"Meilleure validation loss: {min(history['val_loss']):.4f}")


## 6. Évaluation du Modèle

Évaluons les performances sur le set de test.


In [ ]:
# Évaluation sur le test set
test_loss, test_acc, test_predictions, test_labels = eval_model(model, test_loader, device)

print("="*70)
print("ÉVALUATION SUR LE TEST SET")
print("="*70)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

# Rapport de classification
print("\n" + "="*70)
print("RAPPORT DE CLASSIFICATION")
print("="*70)
print(classification_report(test_labels, test_predictions, 
                          target_names=['Négatif', 'Positif']))

# Matrice de confusion
cm = confusion_matrix(test_labels, test_predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Négatif', 'Positif'],
            yticklabels=['Négatif', 'Positif'])
plt.title('Matrice de Confusion - DistilBERT Fine-tuned', fontsize=14, fontweight='bold')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.show()


## 7. Prédictions sur Nouvelles Phrases

Testons le modèle fine-tuné sur de nouvelles phrases.


In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    """Prédit le sentiment d'un texte"""
    model.eval()
    
    # Tokenization
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        prediction = torch.argmax(logits, dim=1)
    
    sentiment = "Positif" if prediction.item() == 1 else "Négatif"
    confidence = probabilities[0][prediction.item()].item()
    
    return sentiment, confidence, probabilities[0].cpu().numpy()

# Tester sur de nouvelles phrases
new_texts = [
    "This movie is absolutely amazing! Best film I've seen this year.",
    "Terrible experience. Poor acting and a boring plot.",
    "It was okay, nothing special but not bad either.",
    "Outstanding performance by all actors. Highly recommended!",
    "Waste of time and money. Very disappointed.",
    "Brilliant cinematography and excellent storytelling.",
]

print("="*70)
print("PRÉDICTIONS SUR DE NOUVELLES PHRASES")
print("="*70)

for text in new_texts:
    sentiment, confidence, probs = predict_sentiment(text, model, tokenizer, device)
    print(f"\nTexte: {text}")
    print(f"Sentiment prédit: {sentiment} (Confiance: {confidence:.4f})")
    print(f"Probabilités: Négatif={probs[0]:.4f}, Positif={probs[1]:.4f}")
    print("-" * 70)


## 8. Comparaison avec Approches Classiques

Comparons les transformers avec des approches classiques (TF-IDF + Logistic Regression).


In [ ]:
# Approche classique : TF-IDF + Logistic Regression
print("Entraînement d'un modèle classique (TF-IDF + Logistic Regression)...")

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)
X_val_tfidf = tfidf_vectorizer.transform(val_texts)
X_test_tfidf = tfidf_vectorizer.transform(test_texts)

# Logistic Regression
lr_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
lr_model.fit(X_train_tfidf, train_labels)

# Prédictions
train_pred_lr = lr_model.predict(X_train_tfidf)
val_pred_lr = lr_model.predict(X_val_tfidf)
test_pred_lr = lr_model.predict(X_test_tfidf)

# Métriques
train_acc_lr = accuracy_score(train_labels, train_pred_lr)
val_acc_lr = accuracy_score(val_labels, val_pred_lr)
test_acc_lr = accuracy_score(test_labels, test_pred_lr)

print("\n" + "="*70)
print("COMPARAISON DES MÉTHODES")
print("="*70)
print(f"\nTF-IDF + Logistic Regression:")
print(f"  Train Accuracy: {train_acc_lr:.4f} ({train_acc_lr*100:.2f}%)")
print(f"  Validation Accuracy: {val_acc_lr:.4f} ({val_acc_lr*100:.2f}%)")
print(f"  Test Accuracy: {test_acc_lr:.4f} ({test_acc_lr*100:.2f}%)")

print(f"\nDistilBERT Fine-tuned:")
print(f"  Train Accuracy: {history['train_acc'][-1]:.4f} ({history['train_acc'][-1]*100:.2f}%)")
print(f"  Validation Accuracy: {history['val_acc'][-1]:.4f} ({history['val_acc'][-1]*100:.2f}%)")
print(f"  Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")

# Visualisation
comparison_data = {
    'Méthode': ['TF-IDF + LR', 'TF-IDF + LR', 'TF-IDF + LR', 'DistilBERT', 'DistilBERT', 'DistilBERT'],
    'Set': ['Train', 'Validation', 'Test', 'Train', 'Validation', 'Test'],
    'Accuracy': [train_acc_lr, val_acc_lr, test_acc_lr, 
                 history['train_acc'][-1], history['val_acc'][-1], test_acc]
}

comparison_df = pd.DataFrame(comparison_data)

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(3)
width = 0.35

methods = ['TF-IDF + LR', 'DistilBERT']
train_accs = [train_acc_lr, history['train_acc'][-1]]
val_accs = [val_acc_lr, history['val_acc'][-1]]
test_accs = [test_acc_lr, test_acc]

x_pos = np.arange(len(methods))
width = 0.25

ax.bar(x_pos - width, train_accs, width, label='Train', alpha=0.8)
ax.bar(x_pos, val_accs, width, label='Validation', alpha=0.8)
ax.bar(x_pos + width, test_accs, width, label='Test', alpha=0.8)

ax.set_xlabel('Méthode', fontsize=12)
ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Comparaison des Performances', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(methods)
ax.legend()
ax.grid(alpha=0.3, axis='y')
ax.set_ylim([0, 1.1])

plt.tight_layout()
plt.show()


## 9. Extraction d'Embeddings et Visualisation

Visualisons les embeddings extraits du modèle BERT pour comprendre comment il représente le texte.


In [ ]:
# Charger BERT pour extraction d'embeddings
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = bert_model.to(device)
bert_model.eval()

def extract_embeddings(texts, model, tokenizer, device, max_length=128):
    """Extrait les embeddings de [CLS] token"""
    embeddings = []
    
    for text in texts:
        encoding = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # Utiliser le token [CLS] (premier token)
            cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embedding[0])
    
    return np.array(embeddings)

# Extraire les embeddings pour quelques exemples
sample_texts = [
    "I love this movie!",
    "This film is terrible.",
    "Great acting and story.",
    "Boring and predictable.",
    "Excellent cinematography!",
    "Waste of time."
]
sample_labels = [1, 0, 1, 0, 1, 0]

print("Extraction des embeddings BERT...")
embeddings = extract_embeddings(sample_texts, bert_model, bert_tokenizer, device)

print(f"Shape des embeddings: {embeddings.shape}")

# Réduction de dimensionnalité avec PCA pour visualisation
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
embeddings_2d = pca.fit_transform(embeddings)

# Visualisation
plt.figure(figsize=(10, 8))
colors = ['green' if label == 1 else 'red' for label in sample_labels]
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=colors, s=100, alpha=0.7, edgecolors='black')

for i, text in enumerate(sample_texts):
    plt.annotate(text[:20] + '...', (embeddings_2d[i, 0], embeddings_2d[i, 1]), 
                fontsize=9, alpha=0.7)

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)', fontsize=12)
plt.title('Visualisation des Embeddings BERT (PCA)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.legend(handles=[
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Positif'),
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Négatif')
])
plt.tight_layout()
plt.show()


## 10. Utilisation de Modèles Pré-entraînés (Sans Fine-tuning)

Explorons comment utiliser des modèles pré-entraînés directement sans fine-tuning.


In [ ]:
# Pipeline pré-entraîné pour sentiment analysis
print("Test avec pipeline pré-entraîné (sans fine-tuning)...")
pretrained_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    device=0 if torch.cuda.is_available() else -1
)

# Tester sur quelques phrases du test set
print("\n" + "="*70)
print("COMPARAISON: PRÉ-ENTRAÎNÉ vs FINE-TUNÉ")
print("="*70)

test_samples = test_texts[:10]
test_true_labels = test_labels[:10]

for i, (text, true_label) in enumerate(zip(test_samples, test_true_labels)):
    # Pré-entraîné
    pretrained_result = pretrained_pipeline(text[:512])[0]  # Limiter la longueur
    pretrained_label = 1 if pretrained_result['label'] == 'POSITIVE' else 0
    
    # Fine-tuné
    fine_tuned_sentiment, fine_tuned_conf, _ = predict_sentiment(text, model, tokenizer, device)
    fine_tuned_label = 1 if fine_tuned_sentiment == "Positif" else 0
    
    print(f"\n{i+1}. Texte: {text[:80]}...")
    print(f"   Vrai label: {'Positif' if true_label == 1 else 'Négatif'}")
    print(f"   Pré-entraîné: {pretrained_result['label']} ({pretrained_result['score']:.3f}) - {'✓' if pretrained_label == true_label else '✗'}")
    print(f"   Fine-tuné: {fine_tuned_sentiment} ({fine_tuned_conf:.3f}) - {'✓' if fine_tuned_label == true_label else '✗'}")
    print("-" * 70)


## 11. Autres Applications des Transformers

### 11.1. Zero-Shot Classification

Les transformers peuvent classifier du texte dans des catégories sans entraînement spécifique.


In [ ]:
# Zero-shot classification
try:
    classifier = pipeline("zero-shot-classification", device=0 if torch.cuda.is_available() else -1)
    
    text_to_classify = "The movie had excellent acting and a compelling story."
    candidate_labels = ["positive", "negative", "neutral", "exciting", "boring"]
    
    result = classifier(text_to_classify, candidate_labels)
    
    print("="*70)
    print("ZERO-SHOT CLASSIFICATION")
    print("="*70)
    print(f"Texte: {text_to_classify}")
    print(f"\nCatégories candidates: {candidate_labels}")
    print(f"\nRésultats:")
    for label, score in zip(result['labels'], result['scores']):
        print(f"  {label}: {score:.4f}")
    
    # Visualisation
    plt.figure(figsize=(10, 6))
    y_pos = np.arange(len(result['labels']))
    plt.barh(y_pos, result['scores'], alpha=0.7, color='steelblue')
    plt.yticks(y_pos, result['labels'])
    plt.xlabel('Score de confiance', fontsize=12)
    plt.title('Zero-Shot Classification', fontsize=14, fontweight='bold')
    plt.grid(alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Zero-shot classification non disponible: {e}")
    print("Cette fonctionnalité nécessite un modèle spécifique (ex: facebook/bart-large-mnli)")


### 11.2. Question Answering (Réponse aux Questions)

Les transformers peuvent répondre à des questions basées sur un contexte.


In [ ]:
# Question Answering
try:
    qa_pipeline = pipeline("question-answering", device=0 if torch.cuda.is_available() else -1)
    
    context = """
    Transformers are deep learning models that have revolutionized natural language processing.
    They use attention mechanisms to process sequences of text. BERT, GPT, and T5 are popular
    transformer models. BERT is bidirectional and great for understanding, while GPT is
    unidirectional and excellent for text generation.
    """
    
    questions = [
        "What are transformers?",
        "Which transformer is good for text generation?",
        "What mechanism do transformers use?"
    ]
    
    print("="*70)
    print("QUESTION ANSWERING")
    print("="*70)
    print(f"Contexte:\n{context.strip()}\n")
    
    for question in questions:
        result = qa_pipeline(question=question, context=context)
        print(f"Question: {question}")
        print(f"Réponse: {result['answer']}")
        print(f"Score: {result['score']:.4f}")
        print("-" * 70)
        
except Exception as e:
    print(f"Question answering non disponible: {e}")


## 12. Sauvegarde et Chargement du Modèle

Sauvegardons notre modèle fine-tuné pour une utilisation ultérieure.


In [ ]:
# Sauvegarder le modèle et le tokenizer
save_directory = "./sentiment_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Modèle sauvegardé dans: {save_directory}")

# Pour charger le modèle plus tard:
# from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
# loaded_model = DistilBertForSequenceClassification.from_pretrained(save_directory)
# loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)


## 13. Résumé et Conclusions

### 13.1. Avantages des Transformers

**Par rapport aux approches classiques (TF-IDF, RNN, LSTM) :**
- **Meilleures performances** : Surtout sur des tâches complexes
- **Contexte bidirectionnel** : BERT comprend le contexte dans les deux directions
- **Transfer Learning** : Modèles pré-entraînés réutilisables
- **Scalabilité** : Fonctionne bien avec de grandes quantités de données

**Par rapport aux RNN/LSTM :**
- **Parallélisation** : Entraînement plus rapide
- **Long-range dependencies** : Meilleure capture des dépendances à long terme
- **Attention mechanism** : Comprend quelles parties du texte sont importantes

### 13.2. Inconvénients et Limitations

- **Coût computationnel** : Plus lent et gourmand en mémoire que les méthodes classiques
- **Données d'entraînement** : Nécessite beaucoup de données pour le pré-entraînement
- **Interprétabilité** : Plus difficile à interpréter qu'un modèle linéaire
- **Taille des modèles** : Modèles très volumineux (millions de paramètres)

### 13.3. Quand Utiliser les Transformers ?

**Utilisez les transformers si :**
- Vous avez accès à des ressources computationnelles suffisantes
- Vous travaillez sur des tâches complexes (traduction, QA, etc.)
- Vous avez des données textuelles abondantes
- La performance est critique

**Utilisez des méthodes classiques si :**
- Vous avez des contraintes de ressources
- Votre dataset est petit
- Vous avez besoin d'interprétabilité
- Les performances acceptables sont suffisantes

### 13.4. Modèles Recommandés

- **Pour classification de texte** : DistilBERT (rapide) ou BERT (plus précis)
- **Pour génération de texte** : GPT-2, GPT-3
- **Pour traduction** : mBART, T5
- **Pour question answering** : BERT, RoBERTa
- **Pour embeddings** : Sentence-BERT

### 13.5. Prochaines Étapes

1. **Optimisation** : Hyperparameter tuning (learning rate, batch size, epochs)
2. **Data Augmentation** : Back-translation, synonym replacement
3. **Ensemble Methods** : Combiner plusieurs modèles
4. **Domain Adaptation** : Fine-tuning sur un domaine spécifique
5. **Model Compression** : Quantization, distillation pour déploiement


## 14. Exercices Complémentaires (Optionnel)

### Exercice 1 : Fine-tuning sur un Autre Dataset
- Téléchargez un autre dataset de classification (ex: AG News, 20 Newsgroups)
- Fine-tunez DistilBERT ou BERT sur ce nouveau dataset
- Comparez les performances avec notre modèle actuel

### Exercice 2 : Hyperparameter Tuning
- Testez différents learning rates (1e-5, 2e-5, 5e-5)
- Variez la taille des batches (8, 16, 32)
- Testez différents nombres d'epochs
- Identifiez les meilleurs hyperparamètres

### Exercice 3 : Comparaison de Modèles
- Comparez BERT, DistilBERT, et RoBERTa sur la même tâche
- Mesurez le temps d'entraînement et les performances
- Analysez les trade-offs

### Exercice 4 : Analyse de l'Attention
- Visualisez les weights d'attention pour comprendre ce sur quoi le modèle se concentre
- Comparez l'attention entre différents exemples
- Interprétez les patterns d'attention

### Exercice 5 : Multi-class Classification
- Adaptez le modèle pour une classification multi-classes (3+ classes)
- Utilisez un dataset comme AG News ou 20 Newsgroups
- Comparez avec des approches classiques
